In [2]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import processing_netcdf as pcdf
import geopandas as gpd
from geopandas import GeoDataFrame
import shapely.geometry 
import numpy as np
from shapely import geometry as gmty
from geofeather import to_geofeather, from_geofeather
import glob
import os
import pyarrow
from xclim import ensembles as ens
from xclim import subset

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import matplotlib.pyplot as plt

In [4]:
folder = "/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/"

# Select variable

In [5]:
#Variable 
variable85="rcp85_tg_mean_annual"
variable45="rcp45_tg_mean_annual"
#variable="rcp[48]5_tg_mean_annual"

files85 = glob.glob(folder+"*"+variable85+".nc")
files45 = glob.glob(folder+"*"+variable45+".nc")
#ex: ACCESS1-3_rcp45_tn_mean_annual.nc
files85
files45


['/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/ACCESS1-3_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/BNU-ESM_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CanESM2_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CMCC-CMS_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/GFDL-ESM2M_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/HadGEM2-CC_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/INM-CM4_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5A-LR_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5B-LR_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/MPI-ESM-LR_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/NorESM1-M_rcp45_tg_mean_annual.nc']

# FOR RCP45 

In [6]:
dsEns45= ens.create_ensemble(files45)
#dsEnsSeas45 = ens.create_ensemble(filesSeas)
dsEns45


<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416, realization: 11, time: 151)
Coordinates:
  * lon          (lon) float32 -89.04521 -88.96188 ... -54.54659 -54.46326
  * time         (time) datetime64[ns] 1950-01-01 1951-01-01 ... 2100-01-01
  * lat          (lat) float32 66.62331 66.53998 66.45665 ... 40.12437 40.04104
  * realization  (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    tg_mean      (realization, time, lat, lon) float32 dask.array<chunksize=(1, 10, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.

## Get the percentiles

In [7]:
# figure
#dsEns45 = dsEns45.assign_coords({'region':mask45})
# spatial means - grouby region mask
#out45 = dsEns45.groupby(dsEns45.region).mean()
#print(out45)
# Calculate ensembles percentiles on regional means
perctiles45 = ens.ensemble_percentiles(dsEns45)
perctiles45

<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416, realization: 11, time: 151)
Coordinates:
  * lon          (lon) float32 -89.04521 -88.96188 ... -54.54659 -54.46326
  * time         (time) datetime64[ns] 1950-01-01 1951-01-01 ... 2100-01-01
  * lat          (lat) float32 66.62331 66.53998 66.45665 ... 40.12437 40.04104
  * realization  (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    tg_mean_p10  (time, lat, lon) float32 dask.array<chunksize=(10, 320, 416), meta=np.ndarray>
    tg_mean_p50  (time, lat, lon) float32 dask.array<chunksize=(10, 320, 416), meta=np.ndarray>
    tg_mean_p90  (time, lat, lon) float32 dask.array<chunksize=(10, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.

## Convert in Dataframe

In [8]:
dfper145 = perctiles45.to_dataframe().dropna()
dfper45 = perctiles45.drop('realization').to_dataframe().dropna()
dfmean45 = dfper45.reset_index()

KeyboardInterrupt: 

In [ ]:
dfmean45

## Cut by region

In [ ]:
# TO-DO 
- Cut by region
- Continue with below operatons within the loop for each region for rcp45, rcp85 and 

## Get mean by year

In [ ]:
dfmean452 = dfmean45.groupby(["time"]).mean()

## 4th degre polynomium

In [ ]:
def polysmooth(years, y, degree):
    coef = np.polyfit(years, y, degree)
    return np.polyval(coef,years)

In [ ]:
years = dfmean452.reset_index().time.dt.year
#years
#years.values
df452 = dfmean452.reset_index()
df452

In [ ]:
rp1 = polysmooth(years,dfmean452.tg_mean_p10,4)
rp5 = polysmooth(years,dfmean452.tg_mean_p50,4)
rp9 = polysmooth(years,dfmean452.tg_mean_p90,4)

## Create DF to plot TimeSeries graph in R

In [ ]:
df45 = pd.DataFrame()
df45["time"] = years.values
df45["tg_mean_p10"] = rp1
df45["tg_mean_p50"] = rp5
df45["tg_mean_p90"] = rp9
df45["tg_mean_p10"] = round(df45["tg_mean_p10"]-273.15, 2)
df45["tg_mean_p50"] = round(df45["tg_mean_p50"]-273.15, 2)
df45["tg_mean_p90"] = round(df45["tg_mean_p90"]-273.15, 2)

In [ ]:
df45

### TODO:
- This dataframe should include both models (RCP 4.5 nad 8.5) and observations (historic)
- I should add labels and similar colors into the shiny graph

In [ ]:
df.to_csv("p4tgmean.csv")

In [ ]:
plt.plot(years, rp5, markersize='10', color='red')
plt.fill_between(years, polysmooth(years, dfmean2.tg_mean_p10- 273.15,4), polysmooth(years, dfmean2.tg_mean_p90- 273.15,4), alpha=0.3, color='red')

In [ ]:
dfmean2.to_csv("timeseries.csv")

### NOTE: this example was for the whole region. 
### TODO: 
- Split DF by regions and get same graph for each region
- Convert degrees in Celsius